In [1]:
import subprocess
import os

In [2]:
sample_names = {}
with open('SRP258158.metadata.txt', 'r') as f, open('02_download_srr.ipynb.samples.txt', 'w') as fout, open('02_download_srr.ipynb.samples.small.txt', 'w') as fout_small:
    fout.write('Name\tR1\tR2\n')
    fout_small.write('Name\tR1\tR2\n')
    first_line = f.readline()
    for line in f:
        line_els = line.strip().split('\t')
        srr = line_els[0]
        sample = line_els[-1]
        sample_counter = 1
        if 'control' in sample.lower():
            sample_name = f'Control_{sample_counter}'
            while sample_name in sample_names:
                sample_counter += 1
                sample_name = f'Control_{sample_counter}'
            sample_names[sample_name] = srr
        else:
            sample_name = f'Treated_{sample_counter}'
            while sample_name in sample_names:
                sample_counter += 1
                sample_name = f'Treated_{sample_counter}'
            sample_names[sample_name] = srr

        if not os.path.exists(f'{srr}_1.fastq.gz'):
            print(f'Downloading {srr}...')

            subprocess.check_output(
                'conda run  --no-capture-output -n kc_sratools fastq-dump --gzip --split-3 ' + srr,
                stderr=subprocess.DEVNULL, shell=True
            )
        else:
            print(f'{srr} already exists, skipping download.')

        for read_num in ["1","2"]:
            subprocess.check_output(
                f'rm -f {srr}_{read_num}.head.fastq.gz',
                stderr=subprocess.DEVNULL, shell=True
            )
            subprocess.check_output(
                f'gunzip -c {srr}_{read_num}.fastq.gz | head -n 75000 > {srr}_{read_num}.head.fastq',
                stderr=subprocess.DEVNULL, shell=True
            )
            subprocess.check_output(
                f'gzip {srr}_{read_num}.head.fastq',
                stderr=subprocess.DEVNULL, shell=True
            )
        fout.write(f'{sample_name}\t{srr}_1.fastq.gz\t{srr}_2.fastq.gz\n')
        fout_small.write(f'{sample_name}\t{srr}_1.head.fastq.gz\t{srr}_2.head.fastq.gz\n')

SRR11612417 already exists, skipping download.


SRR11612416 already exists, skipping download.
SRR11612415 already exists, skipping download.
SRR11612420 already exists, skipping download.
SRR11612419 already exists, skipping download.
SRR11612418 already exists, skipping download.


In [3]:
if not os.path.exists('samples.small.txt'):
    os.symlink('02_download_srr.ipynb.samples.small.txt', 'samples.small.txt')